# SymPy Users Guide
http://docs.sympy.org/latest/guide.html

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from sympy import Symbol

## SymPy’s Architecture

### Basics

All symbolic things are implemented using subclasses of the ***Basic*** class. First, you need to create ***symbols*** using Symbol("x") or numbers using Integer(5) or Float(34.3). Then you construct the ***expression*** using any class from SymPy. For example Add(Symbol("a"), Symbol("b")) gives an instance of the Add class. You can call all methods, which the particular class supports.

For easier use, there is a syntactic sugar for expressions like:
> cos(x) + 1 

is equal to 
> cos(x).__add__(1) 

is equal to 
> Add(cos(x), Integer(1))

or

> 2/cos(x) 

is equal to 
> cos(x).__rdiv__(2)  

is equal to 
>Mul(Rational(2), Pow(cos(x), Rational(-1))).

So, you can write normal expressions using python arithmetics like this:

In [4]:
a = Symbol('a')
b = Symbol('b')
e = (a + b)**2
e

(a + b)**2

but from the SymPy point of view, we just need the classes ***Add, Mul, Pow, Rational, Integer***.

## Automatic evaluation to canonical form
For computation, all expressions need to be in a canonical form, this is done during the creation of the particular instance and only inexpensive operations are performed, necessary to put the expression in the canonical form. So the canonical form doesn’t mean the simplest possible expression. The exact list of operations performed depend on the implementation. Obviously, the definition of the canonical form is arbitrary, the only requirement is that ***all equivalent expressions must have the same canonical form***. We tried the conversion to a canonical (standard) form to be as fast as possible and also in a way so that the result is what you would write by hand - so for example `b*a + -4 + b + a*b + 4 + (a + b)**2 becomes 2*a*b + b + (a + b)**2`.

Whenever you construct an expression, for example Add(x, x), the Add.__new__() is called and it determines what to return. In this case:

In [5]:
from sympy import Add
from sympy.abc import x
e = Add(x, x)
e

2*x

In [6]:
type(e)

sympy.core.mul.Mul

In [7]:
e.args

(2, x)

e is actually an instance of Mul(2, x), because Add.__new__() returned Mul.

## Comparisons
Expressions can be compared using a regular python syntax:

In [8]:
from sympy.abc import x, y
x + y == y + x

True

In [9]:
x + y == y - x

False

We made the following decision in SymPy: a = Symbol("x") and another b = Symbol("x") (with the same string “x”) is the same thing, i.e a == b is True. We chose a == b, because it is more natural; exp(x) == exp(x) is also True for the same instance of x but different instances of exp, so we chose to have exp(x) == exp(x) even for different instances of x.

Sometimes, you need to have a unique symbol, for example as a temporary one in some calculation, which is going to be substituted for something else at the end anyway. This is achieved using Dummy("x"). So, to sum it up:

In [10]:
from sympy import Symbol, Dummy
Symbol("x") == Symbol("x")

True

In [11]:
Dummy("x") == Dummy("x")

False

## Debugging
Starting with 0.6.4, you can turn on/off debug messages with the environment variable SYMPY_DEBUG, which is expected to have the values True or False. For example, to turn on debugging, you would issue:

> [user@localhost]: SYMPY_DEBUG=True ./bin/isympy

## Functionality
There are no given requirements on classes in the library. For example, if they don’t implement the fdiff() method and you construct an expression using such a class, then trying to use the Basic.series() method will raise an exception of not finding the fdiff() method in your class. This “duck typing” has an advantage that you just implement the functionality which you need.

You can define the cos class like this:
```
class cos(Function):
    pass
```    
and use it like `1 + cos(x)`, but if you don’t implement the fdiff() method, you will not be able to call `(1 + cos(x)).series()`.

The symbolic object is characterized (defined) by the things which it can do, so implementing more methods like ***fdiff(), subs()*** etc., you are creating a “shape” of the symbolic object. Useful things to implement in new classes are: ***hash()*** (to use the class in comparisons), ***fdiff() ***(to use it in series expansion), ***subs()*** (to use it in expressions, where some parts are being substituted) and ***series()*** (if the series cannot be computed using the general Basic.series() method). When you create a new class, don’t worry about this too much - just try to use it in your code and you will realize immediately which methods need to be implemented in each situation.

***All objects in sympy are immutable*** - in the sense that ***any operation just returns a new instance*** (it can return the same instance only if it didn’t change). This is a common mistake to change the current instance, like self.arg = self.arg + 1 (wrong!). Use arg = self.arg + 1; return arg instead. The object is immutable in the sense of the symbolic expression it represents. It can modify itself to keep track of, for example, its hash. Or it can recalculate anything regarding the expression it contains. But ***the expression cannot be changed***. So you can pass any instance to other objects, because you don’t have to worry that it will change, or that this would break anything.

## Conclusion
Above are the main ideas behind SymPy that we try to obey. The rest depends on the current implementation and may possibly change in the future. The point of all of this is that ***the interdependencies inside SymPy should be kept to a minimum***. If one wants to add new functionality to SymPy, all that is necessary is to ***create a subclass of Basic*** and implement what you want.

## Functions
How to create a new function with one variable:

In [12]:
from sympy import Basic, Function

class sign(Function):

    nargs = 1

    @classmethod
    def eval(cls, arg):
        if isinstance(arg, Basic.NaN):
            return S.NaN
        if isinstance(arg, Basic.Zero):
            return S.Zero
        if arg.is_positive:
            return S.One
        if arg.is_negative:
            return S.NegativeOne
        if isinstance(arg, Basic.Mul):
            coeff, terms = arg.as_coeff_mul()
            if not isinstance(coeff, Basic.One):
                return cls(coeff) * cls(Basic.Mul(*terms))

    is_finite = True

    def _eval_conjugate(self):
        return self

    def _eval_is_zero(self):
        return isinstance(self[0], Basic.Zero)

and that’s it. The \_eval\_* functions are called when something is needed. The eval is called when the class is about to be instantiated and it should return either some simplified instance of some other class or if the class should be unmodified, return None (see core/function.py in Function.__new__ for implementation details). See also tests in sympy/functions/elementary/tests/test_interface.py that test this interface. You can use them to create your own new functions.

The applied function sign(x) is constructed using
>sign(x)

both inside and outside of SymPy. Unapplied functions sign is just the class itself:
>sign

both inside and outside of SymPy. This is the current structure of classes in SymPy:
```
class BasicType(type):
    pass
class MetaBasicMeths(BasicType):
    ...
class BasicMeths(AssumeMeths):
    __metaclass__ = MetaBasicMeths
    ...
class Basic(BasicMeths):
    ...
class FunctionClass(MetaBasicMeths):
    ...
class Function(Basic, RelMeths, ArithMeths):
    __metaclass__ = FunctionClass
    ...
```
The exact names of the classes and the names of the methods and how they work can be changed in the future.

This is how to create a function with two variables:

In [13]:
class chebyshevt_root(Function):
    nargs = 2

    @classmethod
    def eval(cls, n, k):
        if not 0 <= k < n:
            raise ValueError("must have 0 <= k < n")
        return cos(S.Pi*(2*k + 1)/(2*n))

Here it’s how to define a derivative of the function:

In [14]:
from sympy import Function, sympify, cos
class my_function(Function):
    nargs = 1
    
    def fdiff(self, argindex = 1):
        return cos(self.args[0])
    

    @classmethod
    def eval(cls, arg):
        arg = sympify(arg)
        if arg == 0:
            return sympify(0)

So guess what this my_function is going to be? Well, it’s derivative is ***cos*** and the function value at 0 is 0, but let’s pretend we don’t know:

In [15]:
from sympy import pprint
pprint(my_function(x).series(x, 0, 10))

     3     5     7       9           
    x     x     x       x       ⎛ 10⎞
x - ── + ─── - ──── + ────── + O⎝x  ⎠
    6    120   5040   362880         


Looks familiar indeed:

In [16]:
from sympy import sin
pprint(sin(x).series(x, 0, 10))

     3     5     7       9           
    x     x     x       x       ⎛ 10⎞
x - ── + ─── - ──── + ────── + O⎝x  ⎠
    6    120   5040   362880         


Let’s try a more complicated example. Let’s define the derivative in terms of the function itself:

In [17]:
class what_am_i(Function):
    nargs = 1
    
    def fdiff(self, argindex = 1):
        return 1 - what_am_i(self.args[0])**2
    
    @classmethod
    def eval(cls, arg):
        arg = sympify(arg)
        if arg == 0:
            return sympify(0)

So what is what_am_i? Let’s try it:

In [18]:
pprint(what_am_i(x).series(x, 0, 10))

     3      5       7       9         
    x    2⋅x    17⋅x    62⋅x     ⎛ 10⎞
x - ── + ──── - ───── + ───── + O⎝x  ⎠
    3     15     315     2835         


Well, it’s tanh:

In [19]:
from sympy import tanh
pprint(tanh(x).series(x, 0, 10))

     3      5       7       9         
    x    2⋅x    17⋅x    62⋅x     ⎛ 10⎞
x - ── + ──── - ───── + ───── + O⎝x  ⎠
    3     15     315     2835         


The new functions we just defined are regular SymPy objects, you can use them all over SymPy, e.g.:

In [20]:
from sympy import limit
limit(what_am_i(x)/x, x, 0)

1

## Common tasks
Please use the same way as is shown below all across SymPy.

### accessing parameters:

In [21]:
from sympy import sign, sin
from sympy.abc import x, y, z

e = sign(x**2)
e.args

(x**2,)

In [22]:
e.args[0]

x**2

Number arguments (in Adds and Muls) will always be the first argument;
other arguments might be in arbitrary order:

In [23]:
(1 + x + y*z).args[0]

1

In [24]:
(1 + x + y*z).args[1] in (x, y*z)

True

In [25]:
(y*z).args

(y, z)

In [26]:
sin(y*z).args

(y*z,)

Never use internal methods or variables, prefixed with “_” (example: don’t use _args, use .args instead).

### testing the structure of a SymPy expression

Applied functions:

In [27]:
from sympy import sign, exp, Function
e = sign(x**2)
e

sign(x**2)

In [28]:
isinstance(e, sign)

True

In [29]:
isinstance(e, exp)

False

In [30]:
isinstance(e, Function)

True

So e is a sign(z) function, but not exp(z) function.

Unapplied functions:

In [31]:
from sympy import sign, exp, FunctionClass
e = sign
e

sign

In [32]:
f = exp
f

exp

In [33]:
g = Add
g

sympy.core.add.Add

In [34]:
isinstance(e, FunctionClass)

True

In [35]:
isinstance(f, FunctionClass)

True

In [36]:
isinstance(g, FunctionClass)

False

In [37]:
g is Add

True

In [41]:
g.__mro__

(sympy.core.add.Add,
 sympy.core.expr.Expr,
 sympy.core.operations.AssocOp,
 sympy.core.basic.Basic,
 sympy.core.evalf.EvalfMixin,
 object)

So e and f are functions, g is not a function.